In [1]:
#XGBoost
import pandas as pd
import numpy as np

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
import xgboost as xgb

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
%matplotlib inline

In [8]:
from os.path import dirname, join
from pathlib import Path
ROOT_DIR = dirname(os.path.abspath('ARCH7211-SPRING2021')) # This is your Project Root
data_PATH = join(ROOT_DIR,"data","room_info.csv")  # requires `import os`

# input data
data = pd.read_csv(r'C:\Users\milad\Documents\GitHub\arch7211-spring2021\data\room_info.csv')

In [9]:
exclusion = ["Undefined","UserDefined","Other"]
merge = {
    "Sauna": "Bath",
    "Den":'LivingRoom',
    "Library": "Office",
    "DressingRoom":'Closet'
}

data["type_cleaned"] = data["type"].apply(lambda x: merge.get(x,x))
data["exclude"] = ~data["type"].isin(exclusion)

In [10]:
data_exclude = data[~data["type"].isin(exclusion)]

In [11]:
data_exclude.groupby("type_cleaned",as_index=False).count().sort_values("type_cleaned")

,type_cleaned,path,type,classes,floor_index,num_sides,area,proportion_floor_area,perimeter,compactness,...,contains_Sink,contains_SpaceForAppliance,contains_Stove,contains_Toilet,contains_TumbleDryer,contains_Urinal,contains_WallCabinet,contains_WashingMachine,contains_WaterTap,exclude
0,Alcove,164,164,164,164,164,164,164,164,164,...,164,164,164,164,164,164,164,164,164,164
1,Attic,93,93,93,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93
2,Basement,51,51,51,51,51,51,51,51,51,...,51,51,51,51,51,51,51,51,51,51
3,Bath,7288,7288,7288,7288,7288,7288,7288,7288,7288,...,7288,7288,7288,7288,7288,7288,7288,7288,7288,7288
4,Bedroom,7993,7993,7993,7993,7993,7993,7993,7993,7993,...,7993,7993,7993,7993,7993,7993,7993,7993,7993,7993
5,CarPort,279,279,279,279,279,279,279,279,279,...,279,279,279,279,279,279,279,279,279,279
6,Closet,3041,3041,3041,3041,3041,3041,3041,3041,3041,...,3041,3041,3041,3041,3041,3041,3041,3041,3041,3041
7,Dining,954,954,954,954,954,954,954,954,954,...,954,954,954,954,954,954,954,954,954,954
8,DraughtLobby,1667,1667,1667,1667,1667,1667,1667,1667,1667,...,1667,1667,1667,1667,1667,1667,1667,1667,1667,1667
9,Elevated,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100


In [13]:
#Clean input data from the target data
# define features and predictions
X = data_exclude.drop(['path','type','classes','open_to','door_to','contains','type_cleaned','exclude'], axis =1)
y = data_exclude['type']
#Split the data into training data, and test data 
X_train , X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3)

In [10]:
clf = GradientBoostingClassifier(n_estimators=100).fit(X_train, y_train)
clf.score(X_test,y_test)

0.8782297965915338

In [19]:
#XGBOOST
param_dist = {'objective':'binary:logistic', 'n_estimators':200}

clf = xgb.XGBClassifier(**param_dist)
clf.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric= 'mlogloss',
        verbose=True)
clf_pred = clf.predict(X_test)
# evals_result = clf.evals_result()
print( classification_report(y_test,clf_pred))

[0]	validation_0-mlogloss:1.08151	validation_1-mlogloss:1.12557
[1]	validation_0-mlogloss:0.89958	validation_1-mlogloss:0.95732
[2]	validation_0-mlogloss:0.77513	validation_1-mlogloss:0.84179
[3]	validation_0-mlogloss:0.67843	validation_1-mlogloss:0.75387
[4]	validation_0-mlogloss:0.60468	validation_1-mlogloss:0.68820
[5]	validation_0-mlogloss:0.54626	validation_1-mlogloss:0.63837
[6]	validation_0-mlogloss:0.49961	validation_1-mlogloss:0.59825
[7]	validation_0-mlogloss:0.46146	validation_1-mlogloss:0.56650
[8]	validation_0-mlogloss:0.43112	validation_1-mlogloss:0.54226
[9]	validation_0-mlogloss:0.40505	validation_1-mlogloss:0.52209
[10]	validation_0-mlogloss:0.38299	validation_1-mlogloss:0.50575
[11]	validation_0-mlogloss:0.36315	validation_1-mlogloss:0.49202
[12]	validation_0-mlogloss:0.34665	validation_1-mlogloss:0.48109
[13]	validation_0-mlogloss:0.33121	validation_1-mlogloss:0.47124
[14]	validation_0-mlogloss:0.31798	validation_1-mlogloss:0.46394
[15]	validation_0-mlogloss:0.30591	